<a href="https://colab.research.google.com/github/Nell87/drivendata_richter/blob/main/script/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview**

Based on aspects of building location and construction, **our goal is to predict the level of damage to buildings** caused by the 2015 Gorkha earthquake in Nepal.

The data was collected through surveys by Kathmandu Living Labs and the Central Bureau of Statistics, which works under the National Planning Commission Secretariat of Nepal. This survey is one of the largest post-disaster datasets ever collected, containing valuable information on earthquake impacts, household conditions, and socio-economic-demographic statistics.

In [14]:
####    INCLUDES  _______________________________________ #### 
#Loading Libraries:# 
import pandas as pd
import os
import numpy as np

####    READING TRAIN AND TEST DATA _______________________________________ #### 
train= data = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_values.csv")
test = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/test_values.csv")
print(train.shape)

(260601, 39)


# **EDA**
Let's start checking training and test data similarity (if they are not, the performance on test could be really bad)

In [20]:
# Load libraries
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from imblearn.pipeline import make_pipeline as imb_make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE




# We add the label 1 (train) and 0 (test) and merge both datasets
train["check"]=1
test["check"]=0
merged_data = pd.concat([train,test], axis=0)
merged_data[['check', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type','other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']] = merged_data[['check', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type','other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']].apply(lambda x: x.astype('category'))

std_scaler = StandardScaler()
scaled_df = merged_data.copy()

# scaling features
col = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'age', 'area_percentage','count_floors_pre_eq' ,'height_percentage']
features = scaled_df[col]
features = std_scaler.fit_transform(features.values)
scaled_df[col] = features

x = pd.get_dummies(scaled_df.drop('check', axis=1))
y = scaled_df.check

#x= merged_data.iloc[:, :39]
#y= merged_data.iloc[:,39:]

# Create train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test

rf_clf = RandomForestClassifier() 

#Train the model using the training sets y_pred=clf.predict(X_test)
rf_clf.fit(x_train,y_train)




RandomForestClassifier()

In [21]:
y_pred=rf_clf.predict(x_test)


In [22]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7389414913517713


In [ ]:
train_labels = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_labels.csv")
train = train.merge(train_labels, on='building_id')